In [ ]:
from datasets.socofing import SOCOFingGender, SOCOFingFingers, SOCOFingSubjects
from datasets import FPDataset
import numpy as np
from matplotlib import pyplot as plt
from keras import layers, Sequential, models
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

In [ ]:
# Global Config
seed=9
img_dim = (120, 120, 3)
img_height, img_width, img_channels = img_dim
batch_size = 32

# Dataset configuration

AUTOTUNE = tf.data.AUTOTUNE
split_ratio = [0.7, 0.15, 0.15]
shuffle=True
dsConfig = {'batchSize': batch_size, 'parallelTune': AUTOTUNE, 'split': split_ratio, 'inputDim': img_dim, 'seed': seed, 'shuffle': shuffle}

# Model Config
learning_rate = 0.001
epochs_find_best = 10
epochs_best = 100



print(img_dim)
print(img_height, img_width, img_channels)

genderClassNames=['F','M']
fingersClassNames=['R_thumb', 'R_index', 'R_middle', 'R_ring', 'R_little', 'L_thumb', 'L_index', 'L_middle', 'L_ring', 'L_little ']

In [ ]:
SOCOGender = SOCOFingGender(**dict(dsConfig, sampling=FPDataset.UNDER_SAMPLING))
SOCOSubjects = SOCOFingSubjects(**dsConfig)
SOCOFingers = SOCOFingFingers(**dsConfig)

genderDS = SOCOGender.createDatasets()
subjectDS = SOCOSubjects.createDatasets()
fingersDS = SOCOFingers.createDatasets()

Dataset split check

In [ ]:
unionSet = set()
interSet = set()
sets = []
for ds in subjectDS:
    dsSet = set()
    for _, labels in ds.as_numpy_iterator():
        for label in labels:
            parseLabel = label.decode('utf-8')
            dsSet.add(parseLabel)
    sets.append(dsSet)
    unionSet = unionSet.union(dsSet) 
    interSet = interSet.intersection(dsSet) 
    
print('Union of splitted ds subject: {}, intersaction: {}'.format(len(unionSet), len(interSet)))

In [ ]:
dsNames = ['SOCOGender  ', 'SOCOSubjects', 'SOCOFingers ']
splitNames = ['train', 'test ', 'valid']
datasets = [genderDS, subjectDS, fingersDS]

data = []
for ds in datasets:
    data.append([len(i) for i in ds])

data = []
for ds in datasets:
    splitLens = []
    for split in ds:
        splitLen = 0
        for _, labels in split.as_numpy_iterator():
            splitLen+=len(labels)
        splitLens.append(splitLen)
    data.append(splitLens)

row_format ="{:>15}" * (len(dsNames) + 2)
print(row_format.format("", *splitNames, 'total'))
for name, row in zip(dsNames, data):
    print(row_format.format(name, *row, np.sum(row)))

In [ ]:
def display_images_from_dataset(dataset, title='', numOfImages=10, cNames=None):
  plt.figure(figsize=(15,20))
  plt.suptitle(title)
  rows = int(np.ceil(numOfImages/10))
  i=1
  for image, label in dataset:
    for image, label in zip(image, label):
      
      plt.subplot(rows, 10, i)
      plt.axis('off')
      plt.imshow(image, cmap='gray')
      parseLabel = label
      if cNames:
        parseLabel = cNames[tf.argmax(parseLabel)]
      plt.title(parseLabel, fontsize=16)
      if i==numOfImages:
        break
      i+=1
  plt.tight_layout()
  plt.subplots_adjust(wspace=0.1, hspace=0.1)
  plt.show()
  
train_ds, test_ds, val_ds = genderDS

display_images_from_dataset(train_ds, 'Gender - Train', 100, cNames=genderClassNames)

### Classification Layers ###

Set and initialize a classifier for five classes using fully connected layers, ReLu activation function and Dropout to increase regularization

In [ ]:
fc_classifier = Sequential()
fc_classifier.add(layers.Flatten())
fc_classifier.add(layers.Dense(512, activation='relu'))
fc_classifier.add(layers.Dropout(0.4))
fc_classifier.add(layers.Dense(256, activation='relu'))
fc_classifier.add(layers.Dropout(0.3))
fc_classifier.add(layers.Dense(64, activation='relu'))
fc_classifier.add(layers.Dropout(0.2))
fc_classifier.add(layers.Dense(2, activation='softmax'))

In [ ]:
ResNet50 = Sequential(
    tf.keras.applications.ResNet50(include_top=False, input_shape=img_dim, pooling='max',classes=len(genderClassNames), weights='imagenet')
    )
for layer in ResNet50.layers:
        layer.trainable=False

classifier = models.clone_model(fc_classifier)
classifier = fc_classifier.__class__.from_config(fc_classifier.get_config())
ResNet50.add(classifier)
ResNet50.build((None, img_height, img_width, img_channels))
ResNet50.summary()
ResNet50.compile(optimizer=Adam(learning_rate=learning_rate),loss='binary_crossentropy',metrics=['accuracy'])

ResNet50_history = ResNet50.fit(train_ds, validation_data=val_ds, epochs=epochs_find_best)

In [ ]:
history = ResNet50_history
acc_history = history.history['accuracy']
val_acc_history = history.history['val_accuracy']
loss_history = history.history['loss']
val_loss_history = history.history['val_loss']
fig1 = plt.gcf()
plt.plot(acc_history)
plt.plot(val_acc_history)
plt.axis(ymin=0.2,ymax=1)
plt.grid()
plt.title('ResNet50 Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])